## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [49]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tabulate import tabulate

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/willytakasawa/Projects/data-engineering-nanodegree/Data Modeling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [52]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [53]:
# creates a dataframe to view data from the new file created
df = pd.read_csv('event_datafile_new.csv')
df.head(10)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Mitch Ryder & The Detroit Wheels,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",992,Jenny Take A Ride (LP Version),80
1,The Spill Canvas,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",992,The TIde (LP Version),80
2,Mogwai,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",992,Two Rights Make One Wrong,80
3,Spor,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",992,Way Of The Samurai,80
4,DJ Dizzy,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",992,Sexy Bitch,80
5,Erik Hassle,Tegan,F,70,Levine,183.43138,paid,"Portland-South Portland, ME",992,Hurtful,80
6,Phoenix,Ryann,F,2,Smith,207.15057,free,"Palestine, TX",964,Holdin' On Together,92
7,Arctic Monkeys,Ryann,F,3,Smith,175.43791,free,"Palestine, TX",964,Mardy Bum,92
8,tobyMac,Braden,M,0,Parker,229.69424,free,"Youngstown-Warren-Boardman, OH-PA",801,Burn For You,74
9,Jadakiss / Swizz Beatz / OJ Da Juiceman,Martin,M,1,Johnson,191.84281,free,"Minneapolis-St. Paul-Bloomington, MN-WI",996,Who's Real,55


-----

## Part II. Create Tables and Load Data into Apache Cassandra DB

### **1. Config Cassandra Connection**

#### Creating a Cluster

In [54]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [55]:
# Creates keyspace titled app

try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS app
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [56]:
# Set the keyspace created to the session

try:
    session.set_keyspace('app')
except Exception as e:
    print(e)

------

### **2. Problem:**

#### Analytics Team want to answer 3 questions running queries:

* The artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4

* The following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

* Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


#### **2.1 -  Song description heard during specific session**

In [57]:
# creates table song_session_desc considering sessionId and itemInSession as composite primary key (CPK).
# this CPK enables to filter the data and answer question 1.

query = """
CREATE TABLE IF NOT EXISTS song_session_desc (
    sessionId int,
    itemInSession int,
    artist text,
    song_title text,
    song_len float,
    PRIMARY KEY (sessionId, itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [58]:
# read the csv and insert parsed values from each csv line to corresponding columns.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_session_desc (sessionId, itemInSession, artist, song_title, song_len)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [59]:
# select song description to specific sessionId and itemInSession.

query = "SELECT artist, song_title, song_len from song_session_desc WHERE sessionId = 338 AND itemInSession = 4"
try:
    result = session.execute(query)
except Exception as e:
    print(e)

# creates a list of dicts from query result.
data = []
for row in result:
    data.append(
        {
            'Artist': row.artist,
            'Song Title': row.song_title,
            'Song Length': row.song_len
        }
    )

# creates a dataframe to present data.
df = pd.DataFrame(data)

print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+-----------+---------------------------------+---------------+
| Artist    | Song Title                      |   Song Length |
|-----------+---------------------------------+---------------|
| Faithless | Music Matters (Mark Knight Dub) |       495.307 |
+-----------+---------------------------------+---------------+


---------

#### **2.2 -  Playlist heard from user during specific session**

In [60]:
# creates table song_song_playlist_session considering userId and sessionId as composite primary key (CPK) and itemInSession as clustering column.
# clustering column enables to sort the data as requested on question 2.

query = """
CREATE TABLE IF NOT EXISTS song_playlist_session (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    songTitle text,
    user text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)        

In [61]:
# read the csv and insert parsed values from each csv line to corresponding columns.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, songTitle, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1] + " " + line[4])))

In [62]:
# select playlist from specific user and sessionId.

query = "SELECT artist, songTitle, user from song_playlist_session WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession"
try:
    result = session.execute(query)
except Exception as e:
    print(e)

# creates a list of dicts from query result.
data = []
for row in result:
    data.append(
        {
            'Artist': row.artist,
            'Song Title': row.songtitle,
            'User': row.user
        }
    )

# creates a dataframe to present data.
df = pd.DataFrame(data)

print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+-------------------+------------------------------------------------------+-------------+
| Artist            | Song Title                                           | User        |
|-------------------+------------------------------------------------------+-------------|
| Down To The Bone  | Keep On Keepin' On                                   | Sylvie Cruz |
| Three Drives      | Greece 2000                                          | Sylvie Cruz |
| Sebastien Tellier | Kilometer                                            | Sylvie Cruz |
| Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-------------------+------------------------------------------------------+-------------+


------

#### **2.3 -  Users that heard specific song**

In [63]:
# creates table song_song_listeners considering song_title and userId as composite primary key (CPK).
# this CPK enables to filter the data and answer question 3.

query = """
CREATE TABLE IF NOT EXISTS song_listeners (
    song_title text,
    user_id int,
    user text,
    PRIMARY KEY (song_title, user_id)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

In [64]:
# read the csv and insert parsed values from each csv line to corresponding columns.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song_title, user_id, user)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), (line[1] + " " + line[4])))

In [65]:
# select all users which heard specific song.

query = "SELECT user from song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
    result = session.execute(query)
except Exception as e:
    print(e)


# creates a list of dicts from query result.
data = []
for row in result:
    data.append(
        {
            'User': row.user
        }
    )

# creates a dataframe to present data.
df = pd.DataFrame(data)

print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+------------------+
| User             |
|------------------|
| Jacqueline Lynch |
| Tegan Levine     |
| Sara Johnson     |
+------------------+


-----

### **3. End**

In [66]:
# Drop all the tables created during dev stage.

query = "DROP TABLE IF EXISTS song_session_desc"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Close the session and cluster connection¶

In [67]:
session.shutdown()
cluster.shutdown()